# 01: Sparse Data Structures

While Scientific Computing you'll find out that often you carry around matrices where most of the elements are zero.

There is no strict definition regarding the proportion of zero-value elements for a matrix to qualify as sparse but a common criterion is that the number of non-zero elements is roughly equal to the number of rows or columns.

By contrast, if most of the elements are non-zero, the matrix is considered dense.

The number of zero-valued elements divided by the total number of elements ($mn$ for an $m \times n$ matrix) is sometimes referred to as the sparsity of the matrix.


Every tool for Scientific Computing programming has its own way to handle such sparse data structures. Some are better than others. 
Matlab's is not primed for parallel computing, only accepts double precision sparse data, and it only provides one sparse format (could you tell me which one?). These are pretty huge limitations.

Luckily, all these limitations are gone in Python, in fact, Scipy has its own sparse class: https://docs.scipy.org/doc/scipy/tutorial/sparse.html !

Let's explore together sparse data structures!

In [1]:
import numpy as np
import scipy.sparse as sps

In [2]:
m = 7
n = 7
n_nonzero = 8

a = np.array(range(8)).astype(float)
#a[0] = -5
i = np.array([0, 4, 1, 3, 4, 0, 5, 5])
j = np.array([2, 5, 6, 3, 2, 2, 5, 5])

print( a )
print( i )
print( j )

[0. 1. 2. 3. 4. 5. 6. 7.]
[0 4 1 3 4 0 5 5]
[2 5 6 3 2 2 5 5]


# COO Sparse type

The COO sparse data type is the intuitive one. This would be what you would do straight away if you were to implement your own sparse data type.

Simply, this data format consists of three vectors ```i,j,a``` of lengths equal to ```A.count_nonzero()``` (number of nonzero entries of ```A```), such that the kth element ```a[k]``` is in position ```i[k]```, ```j[k]```:

```
A[i[k], j[k]] = a[k]
```.

In [3]:
A_coo = sps.coo_array((a, (i, j)), shape=(m, n))

In [4]:
print( A_coo )
print(' ')
print( A_coo.todense() )

  (0, 2)	0.0
  (4, 5)	1.0
  (1, 6)	2.0
  (3, 3)	3.0
  (4, 2)	4.0
  (0, 2)	5.0
  (5, 5)	6.0
  (5, 5)	7.0
 
[[ 0.  0.  5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  0.  0.]
 [ 0.  0.  4.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. 13.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


Question 01: why are these two different?

In [5]:
print( sps.coo_array((a, (i, j))).shape )
print( A_coo.shape ) # why??

(6, 7)
(7, 7)


Question 02: why does this happen?

In [6]:
print( A_coo.getnnz() )
print( A_coo.count_nonzero() )
print( A_coo.getnnz() )

8
6
6


I''l do it again but pay attention now:

In [7]:
A_coo = sps.coo_array((a, (i, j)), shape=(m, n))
print( A_coo.row )
print( A_coo.col )
print( A_coo.data )
A_coo.count_nonzero()

[0 4 1 3 4 0 5 5]
[2 5 6 3 2 2 5 5]
[0. 1. 2. 3. 4. 5. 6. 7.]


6

But now, magically:

In [8]:
print( A_coo.row )
print( A_coo.col )
print( A_coo.data )

[0 4 3 4 5 1]
[2 2 3 5 5 6]
[ 5.  4.  3.  1. 13.  2.]


# CSR Sparse type

The CSR sparse data type is way less intuitive.

You still have three vectors, ```indices, indptr, data```. Of different lengths this time.

The column indices for the $i$th row are stored in 

```indices[indptr[i]:indptr[i+1]]``` 

and their corresponding values are stored in 

```data[indptr[i]:indptr[i+1]]```.

But why one would ever think of something so ugly? Well, it is a format meant for increase performance of matrix-vector products. 
Let 

In [9]:
A_csr = sps.csr_array((a, (i, j)), shape=(m, n))
v = np.random.randn(m)

be a matrix in csr format and a vector. 
The following are equivalent

In [19]:
f_ref   = A_csr @ v

indices = A_csr.indices
indptr  = A_csr.indptr
data    = A_csr.data
f = np.zeros(m)
for i_ in range(m):
    f[i_] = data[indptr[i_]:indptr[i_+1]] @ v[indices[indptr[i_]:indptr[i_+1]] ]
    
print('The absolute error is equal to %e' % np.linalg.norm(f-f_ref))

The absolute error is equal to 0.000000e+00


In [20]:
ciao = np.arange(9).reshape((3,3))
print( ciao )

[[0 1 2]
 [3 4 5]
 [6 7 8]]


Try implementing the matrix-vector product with COO format if you dare.

Question 03: why this doesn't happen?

In [10]:
print( A_csr.getnnz() )
print( A_csr.count_nonzero() )
print( A_csr.getnnz() )

6
6
6


Question 04: how to read this?

In [11]:
print( A_csr.indptr )
print( A_csr.indices )
print( A_csr.data )

[0 1 2 2 3 5 6 6]
[2 6 3 2 5 5]
[ 5.  2.  3.  4.  1. 13.]


Here's the matrix in dense format

In [12]:
print(A_csr.todense())

[[ 0.  0.  5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  0.  0.]
 [ 0.  0.  4.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. 13.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [17]:
i = 4
print( A_csr.indptr[i] )
print( A_csr.indptr[i+1] )
print(' ')
print( A_csr.indices[A_csr.indptr[i]:A_csr.indptr[i+1]] )
print(' ')
print( A_csr.data[A_csr.indptr[i]:A_csr.indptr[i+1]] )

3
5
 
[2 5]
 
[4. 1.]


# LIL Sparse type

This is also known as the List of Lists sparse data format.
The explanation basically lies in its name, easier to show than say:

In [22]:
A_lil = sps.lil_array(A_coo)
print(A_lil.todense())

[[ 0.  0.  5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  0.  0.]
 [ 0.  0.  4.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0. 13.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


Question 05: how to read this?

In [23]:
print( A_lil.rows )
A_lil.rows[4][1] # entry (4,5) then

[list([2]) list([6]) list([]) list([3]) list([2, 5]) list([5]) list([])]


5

In [24]:
print( A_lil.data )
A_lil.data[4][1] # we still access data like this

[list([5.0]) list([2.0]) list([]) list([3.0]) list([4.0, 1.0])
 list([13.0]) list([])]


1.0

In [25]:
A_lil[4,5] # or like this implicitly calling something under the hood

1.0

What if I sum two different sparse type matrices (A_coo and A_csr)?

In [26]:
C_ = A_coo + A_csr
C_

<7x7 sparse array of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

What if I sum two different sparse type matrices (A_csr and A_lil)?

In [27]:
D_ = A_csr + A_lil
D_

<7x7 sparse array of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

What if I sum two different sparse type matrices (A_coo and A_lil)?

In [28]:
E_ = A_coo + A_lil
E_

<7x7 sparse array of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

Now let's setup some serious tests.

In [29]:
import time

m = 1000 * 1000
n = 1000 * 1000
n_nonzero = 50 * m

a = np.random.randn(n_nonzero)
i = np.random.randint(0,m,size=(n_nonzero))
j = np.random.randint(0,n,size=(n_nonzero))

v = np.random.randn(m)

A_coo = sps.coo_array((a, (i, j)), shape=(m, n))
A_csr = A_coo.tocsr()
A_lil = A_coo.tolil()

Test 1. Matrix-vector product

In [32]:
# launch this few times

t = time.time()
f_coo = A_coo @ v
print('COO matrix-vector product. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
f_csr = A_csr @ v
print('CSR matrix-vector product. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
f_lil = A_lil @ v
print('LIL matrix-vector product. Elapsed time %1.3f seconds.' % (time.time() - t))

COO matrix-vector product. Elapsed time 1.790 seconds.
CSR matrix-vector product. Elapsed time 0.672 seconds.
LIL matrix-vector product. Elapsed time 9.503 seconds.


In [31]:
# launch this few times

t = time.time()
f_coo = A_coo.T @ v
print('Transposed COO matrix-vector product. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
f_csr = A_csr.T @ v
print('Transposed CSR matrix-vector product. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
f_lil = A_lil.T @ v
print('Transposed LIL matrix-vector product. Elapsed time %1.3f seconds.' % (time.time() - t))

Transposed COO matrix-vector product. Elapsed time 1.400 seconds.
Transposed CSR matrix-vector product. Elapsed time 0.568 seconds.
Transposed LIL matrix-vector product. Elapsed time 54.627 seconds.


Test 2. Column slicing

In [33]:
print('0 COO column slicings. Not possible.')

t = time.time()
for k in range( 10 ):
    A_csr[:,[k]]
print('10 CSR column slicings. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
for k in range( 10 ):
    A_lil[:,[k]]
print('10 LIL column slicings. Elapsed time %1.3f seconds.' % (time.time() - t))

print(' ')
print('0 COO row slicings. Not possible.')

t = time.time()
for k in range( 10000 ):
    A_csr[[k],:]
print('10000 CSR row slicings. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
for k in range( 10000 ):
    A_lil[[k],:]
print('10000 LIL row slicings. Elapsed time %1.3f seconds.' % (time.time() - t))

0 COO column slicings. Not possible.
10 CSR column slicings. Elapsed time 8.235 seconds.
10 LIL column slicings. Elapsed time 65.012 seconds.
 
0 COO row slicings. Not possible.
10000 CSR row slicings. Elapsed time 1.480 seconds.
10000 LIL row slicings. Elapsed time 0.516 seconds.


Test 3: Change entries

In [39]:
t = time.time()
for k in range( 100 ):
    k_ = np.random.randint(0,m,1)
    A_coo[[k_],[k_]] = k_
print('COO changed sparsity structure. Elapsed time %1.3f seconds.' % (time.time() - t))
print('COO sparsity structure changing not possible.')

t = time.time()
for k in range( 100 ):
    k_ = np.random.randint(0,m,1)
    A_csr[[k_],[k_]] = k_
print('CSR changed sparsity structure. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
for k in range( 100 ):
    k_ = np.random.randint(0,m,1)
    A_lil[[k_],[k_]] = k_
print('LIL changed sparsity structure. Elapsed time %1.3f seconds.' % (time.time() - t))

TypeError: 'coo_array' object does not support item assignment

Test 4: Matrix summations

In [35]:
t = time.time()
B_coo = A_coo + A_coo.T
print('COO summation. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
B_csr = A_csr + A_csr.T
print('CSR summation. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
B_lil = A_lil + A_lil.T
print('LIL summation. Elapsed time %1.3f seconds.' % (time.time() - t))

COO summation. Elapsed time 24.377 seconds.
CSR summation. Elapsed time 10.450 seconds.
LIL summation. Elapsed time 62.952 seconds.


In [41]:
B_coo

<1000000x1000000 sparse array of type '<class 'numpy.float64'>'
	with 50000000 stored elements in COOrdinate format>

Test 5: Matrix transpositions

In [42]:
t = time.time()
B_coo = A_coo.T # indexes are manipulated, data is NOT copied
print('COO transposition. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
B_coo = A_coo.T.copy() # indexes are manipulated, data is copied
print('COO actual transposition. Elapsed time %1.3f seconds.' % (time.time() - t))

COO transposition. Elapsed time 1.405 seconds.
COO actual transposition. Elapsed time 2.454 seconds.


In [37]:
t = time.time()
B_csr = A_csr.T # indexes are NOT manipulated, data is NOT copied
print('CSR transposition. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
B_csr = A_csr.T.copy() # indexes are manipulated, data is copied
print('CSR actual transposition. Elapsed time %1.3f seconds.' % (time.time() - t))

CSR transposition. Elapsed time 0.040 seconds.
CSR actual transposition. Elapsed time 0.885 seconds.


In [38]:
t = time.time()
B_lil = A_lil.T # everything has to change
print('LIL transposition. Elapsed time %1.3f seconds.' % (time.time() - t))

t = time.time()
B_lil = A_lil.T.copy() # everything has to change but Idk why, slower...! I think A_lil.T is somehow a middle form
print('LIL actual transposition. Elapsed time %1.3f seconds.' % (time.time() - t))

LIL transposition. Elapsed time 45.938 seconds.
LIL actual transposition. Elapsed time 240.830 seconds.


### TO SUM UP:

Advantages of the COO format
- very fast conversion to and from CSR/CSC formats
- permits duplicate entries (see example)

Disadvantages of the COO format
- does not directly support arithmetic operations
- not fast at matrix vector products (consider CSR or CSC)


Advantages of the CSR format
- efficient arithmetic operations CSR + CSR, CSR @ CSR, etc.
- efficient row slicing
- fast matrix vector products

Disadvantages of the CSR format
- slow column slicing operations (consider CSC)
- changes to the sparsity structure are expensive (consider LIL or COO)


Advantages of the LIL format
- supports flexible slicing
- changes to the matrix sparsity structure are efficient

Disadvantages of the LIL format
- arithmetic operations LIL + LIL are slow (consider CSR or CSC)
- slow column slicing (consider CSC)
- slow matrix vector products (consider CSR or CSC)

